In [1]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

def run_chess_test(opening_moves, timeout=240):

    # Join the moves into a single string without list formatting
    opening_moves_str = " ".join(str(move) for move in opening_moves)

    # Define the command and options for running the Robot Framework script
    robot_command = [
        "robot",
        "--variable", f"OPENING_MOVES:{opening_moves_str}",
        "chessML.robot"
    ]

    print("Running command:", robot_command)

    # Start the Robot Framework process
    process = subprocess.Popen(robot_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    try:
        # Capture and print the output in real-time, with a timeout
        stdout, stderr = process.communicate(timeout=timeout)

        print(stdout) 
        if stderr:
            print(stderr)

    except subprocess.TimeoutExpired:
        print("Process timed out; forcing termination.")
        process.kill()  # Forcefully terminate the process after timeout
        stdout, stderr = process.communicate()
        print(stdout)
        if stderr:
            print(stderr)

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        process.terminate()

    print("Process finished and resources cleaned up.\n")

df = pd.read_csv("openingMoves.csv",header=None,comment='#')

# Example list of opening moves to test
#opening_moves = [["e2e4", "g1f3", "f1c4"], ["e2e4", "e7e5", "f1c4"]]
opening_moves = df.values.tolist()

# Create a thread pool executor
with ThreadPoolExecutor(max_workers=8) as executor:
    # Submit tasks to the executor
    futures = [executor.submit(run_chess_test, moves) for moves in opening_moves]

    # Wait for all tasks to complete
    for future in as_completed(futures):
        try:
            future.result()  # Retrieve the result to catch any exceptions
        except Exception as e:
            print(f"An error occurred during execution: {e}")

Running command:Running command: ['robot', '--variable', 'OPENING_MOVES:e2e4 e7e5 f1c4', 'chessML.robot']
 ['robot', '--variable', 'OPENING_MOVES:e2e4 e7e5 g1f3', 'chessML.robot']
Running command: ['robot', '--variable', 'OPENING_MOVES:d2d4 d7d5 c2c4', 'chessML.robot']
Running command: ['robot', '--variable', 'OPENING_MOVES:e2e4 c7c5 g1f3', 'chessML.robot']
Running command: ['robot', '--variable', 'OPENING_MOVES:e2e4 e7e6 d2d4', 'chessML.robot']
Running command: ['robot', '--variable', 'OPENING_MOVES:d2d4 g8f6 c4g6', 'chessML.robot']
Running command: ['robot', '--variable', 'OPENING_MOVES:g1f3 d7d5 g2g3', 'chessML.robot']
Running command: ['robot', '--variable', 'OPENING_MOVES:c2c4 e7e5 g1f3', 'chessML.robot']
Process timed out; forcing termination.Process timed out; forcing termination.
Process timed out; forcing termination.

Process timed out; forcing termination.
chessML                                                                       
Test Initialization                      

In [12]:
import subprocess
import pandas as pd

def run_chess_test_with_visual_game(opening_moves, timeout=240):
    # Convert the opening moves to a format Robot Framework understands
    if not isinstance(opening_moves, list):
        raise ValueError("Expected opening_moves to be a list.")

    # Join the moves into a single string without list formatting
    opening_moves_str = " ".join(str(move) for move in opening_moves)

    # Define the command and options for running the Robot Framework script
    robot_command = [
        "robot",
        "--variable", f"OPENING_MOVES:{opening_moves_str}",
        "chessVisual.robot"
    ]

    # Debugging output to ensure the command is as expected
    print("Running command:", robot_command)

    # Start the Robot Framework process
    process = subprocess.Popen(robot_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    try:
        # Capture and print the output in real-time, with a timeout
        stdout, stderr = process.communicate(timeout=timeout)

        print(stdout)  # Print all captured stdout output
        if stderr:
            print(stderr)  # Print all captured stderr output, if any

    except subprocess.TimeoutExpired:
        print("Process timed out; forcing termination.")
        process.kill()  # Forcefully terminate the process after timeout
        stdout, stderr = process.communicate()  # Capture any remaining output
        print(stdout)
        if stderr:
            print(stderr)

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # Ensure all remaining output is captured and the process is cleaned up
        process.terminate()  # Clean up the process

    print("Process finished and resources cleaned up.\n")

# Load data
df = pd.read_csv('openingMoves.csv', header=None, comment='#')
opening_moves = df.values.tolist()

# Run the test with visualization for the first set of opening moves
run_chess_test_with_visual_game(opening_moves[0])

Running command: ['robot', '--variable', 'OPENING_MOVES:e2e4 e7e5 g1f3', 'chessVisual.robot']
chessVisual                                                                   
Test Initialization                                                   | FAIL |
No keyword with name 'Initialize Game Visual' found.
------------------------------------------------------------------------------
chessVisual                                                           | FAIL |
1 test, 0 passed, 1 failed
Output:  /Users/marcokaferbeck/DataspellProjects/roboticProccessAutomationChess/output.xml
Log:     /Users/marcokaferbeck/DataspellProjects/roboticProccessAutomationChess/log.html
Report:  /Users/marcokaferbeck/DataspellProjects/roboticProccessAutomationChess/report.html

[ ERROR ] Error in file '/Users/marcokaferbeck/DataspellProjects/roboticProccessAutomationChess/chessVisual.robot' on line 3: Importing library 'chess_library.ChessLibrary' failed: TypeError: show_boards_sequentially_simple() missing 1 r

In [11]:
from chess_library import ChessLibrary

from IPython.display import display, clear_output
from PIL import Image
import os
import time

def show_boards_sequentially_simple(output_folder, delay=1.0):
    """
    Displays each PNG file in the output folder one by one with a delay.
    
    Parameters:
    - output_folder (str): Path to the folder containing PNG files.
    - move_count (int): Total number of moves (PNG files) to display.
    - delay (float): Time in seconds to wait between frames.
    """
    # List all files in the output folder
    files = os.listdir(output_folder)

    # Filter out only SVG files
    svg_files = [file for file in files if file.endswith('.svg')]

    # Calculate move count
    move_count = len(svg_files)
    
    for i in range(move_count):
        file_path_png = os.path.join(output_folder, f"move_{i}.png")

        # Check if the file exists
        if not os.path.exists(file_path_png):
            print(f"File not found: {file_path_png}")
            continue

        # Open and display the image
        img = Image.open(file_path_png)
        display(img)

        # Wait for the specified delay, then clear output
        time.sleep(delay)
        clear_output(wait=True)

# Example usage after running a game
chess_library_instance = ChessLibrary()
chess_library_instance.initialize_game_visual()
chess_library_instance.play_full_game_visual(max_turns=100)
show_boards_sequentially_simple('chess_svgs')

In [10]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# Load data
data = pd.read_csv('game_results.csv')
X = data[['move1', 'move2', 'move3', 'bot1', 'bot2']]
y = data['Result']

# Update y to replace any text inside parentheses with an empty string, only for "Draw" results
y = y.str.replace(r"\(.*\)", "", regex=True).str.strip()

# Convert categorical data to numeric for ML
X = pd.get_dummies(X)

# Train a RandomForest with 10-fold cross-validation
model = RandomForestClassifier()
cv_scores = cross_val_score(model, X, y, cv=10, scoring='accuracy')

# Print the mean accuracy and standard deviation
print("Mean accuracy:", cv_scores.mean())
print("Standard deviation:", cv_scores.std())

# Fit the model on the entire dataset
model.fit(X, y)

# Create a sample input for prediction
sample_input = pd.DataFrame({
    'move1': ['e2e4'],
    'move2': ['e7e5'],
    'move3': ['g1f3'],
    'bot1': ['LCZero'],
    'bot2': ['LCZero']
})

# Convert the sample input to the same format as the training data
sample_input = pd.get_dummies(sample_input)
sample_input = sample_input.reindex(columns=X.columns, fill_value=0)

# Predict the result for the sample input
prediction = model.predict(sample_input)

# Print the prediction result
print("Prediction for sample input:", prediction[0])

Mean accuracy: 0.7071428571428571
Standard deviation: 0.18809523809523812
Prediction for sample input: Draw


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load data
data = pd.read_csv('game_results.csv')
X = data[['move1', 'move2', 'move3', 'bot1', 'bot2']]
y = data['Result']

# Update y to replace any text inside parentheses with an empty string, only for "Draw" results
y = y.str.replace(r"\(.*\)", "", regex=True).str.strip()

# Convert categorical data to numeric for ML
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the RandomForestClassifier on the training set
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model on the testing set
test_score = model.score(X_test, y_test)
print("Testing score:", test_score)

# Create a DataFrame to store the performance of each opening
openings_performance = pd.DataFrame(columns=['Opening', 'Wins', 'Losses', 'Draws', 'Total', 'Win Rate'])

# Calculate the performance of each opening
performance_list = []
for opening in data['move1'].unique():
    opening_data = data[data['move1'] == opening]
    wins = opening_data[opening_data['Result'] == 'White wins!'].shape[0]
    losses = opening_data[opening_data['Result'] == 'Black wins!'].shape[0]
    draws = opening_data[opening_data['Result'] == 'Draw'].shape[0]
    total = opening_data.shape[0]
    win_rate = wins / total if total > 0 else 0
    performance_list.append({
        'Opening': opening,
        'Wins': wins,
        'Losses': losses,
        'Draws': draws,
        'Total': total,
        'Win Rate': win_rate
    })

# Convert the list of dictionaries to a DataFrame
openings_performance = pd.DataFrame(performance_list)

# Sort the openings by win rate
openings_performance = openings_performance.sort_values(by='Win Rate', ascending=False)

# Print the ranked openings
print(openings_performance)

Testing score: 0.8125
  Opening  Wins  Losses  Draws  Total  Win Rate
0    e2e4    15       7      0     48  0.312500
1    d2d4     4       4      0     14  0.285714
2    g1f3     2       3      0      7  0.285714
3    c2c4     2       2      0      7  0.285714
